In [1]:
import requests
import time
import pandas as pd
import os

In [ ]:
# 엑셀 파일 불러오기
df = pd.read_excel("C:/Path/병원 풀데이터.xlsx", sheet_name="Sheet1")
#pharmacy_df = pd.read_excel("C:/Path/약국 풀 데이터.xlsx")

In [ ]:
# 설정
app_key = "SK open API app key"
address_col = 'dutyAddr'     # 변환 대상 열 이름

In [4]:
# ---------- 1. 미처리 데이터 불러오기 ----------
#df = pd.read_excel("미처리.xlsx")

In [17]:
# ---------- 2. 지번 주소 변환 함수 ----------
def convert_to_jibun(road_address, api_key):
    url = f"https://business.juso.go.kr/addrlink/addrLinkApi.do"
    params = {
        "confmKey": api_key,
        "currentPage": "1",
        "countPerPage": "1",
        "keyword": road_address,
        "resultType": "json"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data["results"]["common"]["errorCode"] == "0":
            juso_list = data["results"]["juso"]
            if juso_list:
                jibun_addr = juso_list[0].get("jibunAddr", "")
                return jibun_addr if jibun_addr.strip() else None
        else:
            print(f"⚠️ API 오류: {data['results']['common']['errorMessage']}")
            return None
    except Exception as e:
        print(f"❌ 요청 실패: {e}")
        return None

In [ ]:
# ---------- 3. 최대 2만건까지 처리 ----------
# 1. 지번주소 결과를 담을 Series 생성
jibun_series = []

# 2. 주소 변환 및 저장
count = 0
unprocessed_idx = []

for i, addr in enumerate(df[address_col]):
    if count >= 20000:
        jibun_series.append(None)
        continue

    jibun = convert_to_jibun(addr)

    if jibun is not None and jibun.strip() != "":
        jibun_series.append(jibun)
        count += 1
    else:
        jibun_series.append(None)
        unprocessed_idx.append(i)

    time.sleep(0.2)

print(f"✅ 오늘 {count}건 처리 완료")

# 3. Jibun 열을 dutyAddr 바로 오른쪽에 삽입
insert_at = df.columns.get_loc(address_col) + 1
df.insert(insert_at, "Jibun", jibun_series)

In [ ]:
# ---------- 4. 기존 파일 있으면 이어붙이고 없으면 새로 저장 ----------
processed_today = df.drop(index=unprocessed_idx)

if os.path.exists("C:/Path/병원 풀데이터 지번 주소.xlsx"):
    processed_prev = pd.read_excel("C:/Path/병원 풀데이터 지번 주소.xlsx")
    updated_df = pd.concat([processed_prev, processed_today], ignore_index=True)
else:
    updated_df = processed_today

updated_df.to_excel("C:/Path/병원 풀데이터 지번 주소.xlsx", index=False)
print("📄 '병원 풀데이터 지번 주소.xlsx'에 저장 완료")

In [ ]:
# ---------- 4. 기존 처리 완료 파일에 덧붙이기 ----------
processed_today = df.drop(index=unprocessed_idx)

try:
    processed_prev = pd.read_excel("C:/Path/병원 풀데이터 지번 주소.xlsx")
    updated_df = pd.concat([processed_prev, processed_today], ignore_index=True)
except FileNotFoundError:
    updated_df = processed_today

updated_df.to_excel("C:/Path/병원 풀데이터 지번 주소.xlsx", index=False)
print("📄 기존 '병원 풀데이터 지번 주소.xlsx'에 덮어쓰기 저장 완료")

In [16]:
print(f"❓ 미처리 인덱스 개수: {len(unprocessed_idx)}")


❓ 미처리 인덱스 개수: 0


In [15]:
# ---------- 5. 남은 미처리 저장 ----------
if unprocessed_idx:
    chunk_size = 20000
    unprocessed_df = df.iloc[unprocessed_idx].reset_index(drop=True)
    output_dir = os.getcwd()

    for i in range(0, len(unprocessed_df), chunk_size):
        chunk = unprocessed_df.iloc[i:i+chunk_size]
        file_index = i // chunk_size + 1
        file_path = os.path.join(output_dir, f"미처리_{file_index}.xlsx")

        try:
            chunk.to_excel(file_path, index=False)
            if os.path.exists(file_path):
                print(f"✅ 저장 완료: {file_path}")
            else:
                print(f"❌ 저장 실패 (파일 없음): {file_path}")
        except Exception as e:
            print(f"❌ 예외 발생: {file_path}")
            print(e)
else:
    print("🎉 전체 데이터 변환 완료! 더 이상 미처리 없음.")

🎉 전체 데이터 변환 완료! 더 이상 미처리 없음.
